In [1]:
import re
import time
import urllib.request

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

### Crawling
Crawling URLs to access statistical values from each meteorological observation station.

In [2]:
url = "https://www.data.jma.go.jp/obd/stats/etrn/select/prefecture00.php?prec_no=&block_no=&year=&month=&day=&view="
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html, "html.parser")

elements = soup.find_all("area")
area_list = [element["alt"] for element in elements]
area_link_list = [element["href"] for element in elements]

In [3]:
region_list = []
prec_no_list = []
block_no_list = []
station_list = []
station_type_list = []
lat_list = []
lon_list = []
altitude_list = []

for area, area_link in tqdm(zip(area_list, area_link_list), total=len(area_list)):

    pattern = re.compile(r'prec_no=(\d+)')
    match = pattern.search(area_link)
    prec_no = match.group(1) if match else None
    url = "https://www.data.jma.go.jp/obd/stats/etrn/select/" + area_link
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html, "html.parser")
    elements = soup.find_all("area")

    for element in elements:
        
        onmouseover_text = element.get("onmouseover")
        if onmouseover_text:
            args = onmouseover_text.split("'")
            block_no_list.append(args[3])
            if len(args[3])==5:
                station_type_list.append("s1")
            else:
                station_type_list.append("a1")
            station_list.append(args[5])
            lat_list.append([float(args[9]), float(args[11])])
            lon_list.append([float(args[13]), float(args[15])])
            altitude_list.append(float(args[17]))
            prec_no_list.append(prec_no)
            region_list.append(area)

  0%|          | 0/61 [00:00<?, ?it/s]

In [4]:
df_station = pd.DataFrame({
    "region": region_list,
    "prec_no": prec_no_list,
    "block_no": block_no_list,
    "station": station_list,
    "station_type": station_type_list,
    "lat": lat_list,
    "lon": lon_list,
    "altitude": altitude_list
})

df_station["lat"] = df_station["lat"].str[0] + df_station["lat"].str[1]/60
df_station["lon"] = df_station["lon"].str[0] + df_station["lon"].str[1]/60
df_station = df_station.drop_duplicates()

In [5]:
df_station

,region,prec_no,block_no,station,station_type,lat,lon,altitude
0,宗谷地方,11,47401,稚内,s1,45.415000,141.678333,2.8
2,宗谷地方,11,0002,沓形,a1,45.178333,141.138333,14.0
4,宗谷地方,11,0003,浜頓別,a1,45.125000,142.350000,18.0
6,宗谷地方,11,47402,北見枝幸,s1,44.940000,142.585000,6.7
8,宗谷地方,11,0005,歌登,a1,44.841667,142.480000,14.0
...,...,...,...,...,...,...,...,...
3344,沖縄県,91,1586,国頭,a1,26.728333,128.178333,8.0
3346,沖縄県,91,1596,宮城島,a1,26.363333,127.971667,100.0
3348,沖縄県,91,1639,盛山,a1,24.395000,124.245000,31.0
3350,沖縄県,91,1651,渡名喜,a1,26.373333,127.141667,7.0


In [6]:
import plotly.express as px
import plotly.graph_objs as go

fig = px.scatter_geo(df_station,
                     lat="lat",
                     lon="lon",
                     hover_name="station",
                     scope="asia",
                     color="station_type",
                     center={"lat": 36.2048, "lon": 138.2529},
                     projection="mercator")

fig.update_geos(
    lataxis_range=[29, 48],  
    lonaxis_range=[128, 147]
)

fig.update_layout(
    title="AMeDAS points in Japan",
    width=700,
    height=1000,
)

fig.show()


### Scraping
Scraping weather data embedded in tables.

In [7]:
hourly_columns_s1 = ["hour", "local-atmospheric-pressure", "sea-level-pressure", "precipitation", 
                     "temperature", "dew-point-temperature", "vapor-pressure", "humidity", "wind-speed",
                     "wind-direction", "sunshine-duration", "global-solar-radiation", "snowfall", "snow-depth", 
                     "weather", "cloud-cover", "visibility"]

hourly_columns_a1 = ["hour", "precipitation", "temperature", "dew-point-temperature", "vapor-pressure", "humidity", 
                     "wind-speed","wind-direction", "sunshine-duration", "snowfall", "snow-depth"]


daily_columns_s1 = ["day", "avg-local-atmospheric-pressure", "avg-sea-level-pressure", "sum-precipitation", "hourly-max-precipitation", "10min-max-precipitation", 
                    "avg-temperature", "max-temperature", "min-temperature", "avg-humidity", "min-humidity", "avg-wind-speed", "max-wind-speed", "max-wind-direction",
                    "max-gust-wind-speed", "max-gust-wind-direction", "sunshine-duration", "sum-snowfall", "max-snow-depth", "daytime-weather_06-18", "nighttime-weather_18-06"]


daily_columns_a1 = ["day", "sum-precipitation", "hourly-max-precipitation", "10min-max-precipitation", 
                    "avg-temperature", "max-temperature", "min-temperature", "avg-humidity", "min-humidity", "avg-wind-speed", "max-wind-speed", "max-wind-direction",
                    "max-gust-wind-speed", "max-gust-wind-direction", "prevailing-wind-direction", "sunshine-duration", "sum-snowfall", "max-snow-depth"]


monthly_columns_s1 = ["month", "avg-local-atmospheric-pressure", "avg-sea-level-pressure", "sum-precipitation", "daily-max-precipitation", "hourly-max-precipitation", "10min-max-precipitation", 
                     "daily-avg-temperature", "daily-max-temperature", "daily-min-temperature", "max-temperature", "min-temperature", "avg-humidity", "min-humidity",
                     "avg-wind-speed", "max-wind-speed", "max-wind-direction", "max-gust-wind-speed", "max-gust-wind-direction", "sunshine-duration", "avg-global-solar-radiation", 
                     "sum-snowfall", "daily-sum-snowfall", "max-snow-depth", "avg-cloud-cover", "snow-days", "fog-days", "thunderstorm-days"]

monthly_columns_a1 = ["month", "sum-precipitation", "daily-max-precipitation", "hourly-max-precipitation", "10min-max-precipitation", 
                     "daily-avg-temperature", "daily-max-temperature", "daily-min-temperature", "max-temperature", "min-temperature", "avg-humidity", "min-humidity",
                     "avg-wind-speed", "max-wind-speed", "max-wind-direction", "max-gust-wind-speed", "max-gust-wind-direction", "sunshine-duration",
                     "sum-snowfall", "daily-sum-snowfall", "max-snow-depth"]

In [8]:
df_station[df_station["station"]=="東京"]

,region,prec_no,block_no,station,station_type,lat,lon,altitude
1322,東京都,44,47662,東京,s1,35.691667,139.75,25.2


In [9]:
# Tokyo
station_type = "s1"
prec_no = "44"
block_no = "47662"

if station_type == "s1":
    hourly_columns = hourly_columns_s1
    daily_columns = daily_columns_s1
    monthly_columns = monthly_columns_s1
else:
    hourly_columns = hourly_columns_a1
    daily_columns = daily_columns_a1
    monthly_columns = monthly_columns_a1

In [10]:
year = 2023
month = 1
day = 1
table_hourly = pd.read_html(f"https://www.data.jma.go.jp/obd/stats/etrn/view/hourly_{station_type}.php?prec_no={prec_no}&block_no={block_no}&year={year}&month={month}&day={day}")
df_hourly = pd.DataFrame(table_hourly[0])
df_hourly.columns = hourly_columns

table_daily = pd.read_html(f"https://www.data.jma.go.jp/obd/stats/etrn/view/daily_{station_type}.php?prec_no={prec_no}&block_no={block_no}&year={year}&month={month}")
df_daily = pd.DataFrame(table_daily[0])
df_daily.columns = daily_columns

table_monthly = pd.read_html(f"https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_{station_type}.php?prec_no={prec_no}&block_no={block_no}&year={year}")
df_monthly = pd.DataFrame(table_monthly[0])
df_monthly.columns = monthly_columns


In [11]:
df_hourly

,hour,local-atmospheric-pressure,sea-level-pressure,precipitation,temperature,dew-point-temperature,vapor-pressure,humidity,wind-speed,wind-direction,sunshine-duration,global-solar-radiation,snowfall,snow-depth,weather,cloud-cover,visibility
0,1,1017.9,1020.9,--,3.5,-0.5,5.9,75,1.0,北西,NaN,NaN,--,--,NaN,NaN,NaN
1,2,1018.1,1021.1,--,3.2,-1.0,5.7,74,1.3,北北西,NaN,NaN,--,--,NaN,NaN,NaN
2,3,1017.8,1020.8,--,3.4,-1.3,5.5,71,0.9,北北西,NaN,NaN,--,--,NaN,0,25.0
3,4,1018.0,1021.0,--,3.2,-1.7,5.4,70,1.4,北北西,NaN,NaN,--,--,NaN,NaN,NaN
4,5,1018.1,1021.2,--,1.8,-1.1,5.6,81,1.4,西北西,NaN,NaN,--,--,NaN,NaN,NaN
5,6,1018.3,1021.4,--,1.6,-1.8,5.4,78,0.7,西,NaN,NaN,--,--,NaN,0+,25.0
6,7,1018.4,1021.5,--,1.3,-2.1,5.2,78,1.2,西北西,0.0,0.00,--,--,NaN,NaN,NaN
7,8,1018.4,1021.4,--,2.9,-1.5,5.5,73,0.6,南西,0.5,0.25,--,--,NaN,NaN,NaN
8,9,1018.3,1021.3,--,6.4,-0.8,5.8,60,0.9,南南東,1.0,0.90,--,--,NaN,0+,25.0
9,10,1017.9,1020.9,--,8.5,-1.6,5.4,49,1.4,北西,1.0,1.43,--,--,NaN,NaN,NaN


### Visualization

In [12]:
df = pd.DataFrame()

for year in range(2000, 2024):
    table_monthly = pd.read_html(f"https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_{station_type}.php?prec_no={prec_no}&block_no={block_no}&year={year}")
    df_monthly = pd.DataFrame(table_monthly[0])
    df_monthly.columns = monthly_columns
    df_monthly = df_monthly[["month", "daily-avg-temperature"]]
    df_monthly["year"] = year

    df = pd.concat([df, df_monthly])

In [13]:
df

,month,daily-avg-temperature,year
0,1,7.6,2000
1,2,6.0,2000
2,3,9.4,2000
3,4,14.5,2000
4,5,19.8,2000
...,...,...,...
7,8,29.2,2023
8,9,26.7,2023
9,10,18.9,2023
10,11,14.4,2023


In [14]:
heatmap_data = df.pivot(index="year", columns="month", values="daily-avg-temperature")

fig = go.Figure(data=go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale="RdBu_r",
        colorbar=dict(title="Temperature (°C)"),
    ))

fig.update_layout(
    title="Average Temperatures in Tokyo (2000 - 2023)",
    xaxis=dict(title="Month"),
    yaxis=dict(title="Year"),
)

fig.show()